In [1]:
# Test finetuning ssd

In [21]:
import torchvision
import torch.nn as nn
from functools import partial
from torchvision.models.detection import _utils as det_utils
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead
from torchvision.models.detection.ssd import SSDClassificationHead
def finetune_ssd300_vgg16(num_classes):
    # model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)
    model = torchvision.models.detection.ssd300_vgg16(pretrained=True)

    in_channels = det_utils.retrieve_out_channels(model.backbone, (320, 320))
    num_anchors = model.anchor_generator.num_anchors_per_location()
    # norm_layer  = partial(nn.BatchNorm2d, eps=0.001, momentum=0.03)
    # num_classes = 2
    model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)
    return model
def ssdlite320_mobilenet_v3_large(num_classes):
    model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)
    in_channels = det_utils.retrieve_out_channels(model.backbone, (320, 320))
    num_anchors = model.anchor_generator.num_anchors_per_location()
    norm_layer  = partial(nn.BatchNorm2d, eps=0.001, momentum=0.03)
    model.head.classification_head = SSDLiteClassificationHead(in_channels, num_anchors, num_classes,norm_layer)
    return model

In [22]:
# model = finetune_ssd300_vgg16(2)
model = ssdlite320_mobilenet_v3_large(2)

In [23]:
import torch
model.eval()
model(torch.rand(1,3,320,320))

[{'boxes': tensor([[2.8627e+02, 2.9250e+02, 2.9633e+02, 3.0462e+02],
          [2.9010e+02, 2.4721e+00, 2.9895e+02, 1.1678e+01],
          [2.8877e+02, 1.3302e+02, 2.9581e+02, 1.4116e+02],
          [2.8816e+02, 1.4896e+02, 2.9535e+02, 1.5717e+02],
          [2.8708e+02, 2.7733e+02, 2.9701e+02, 2.8850e+02],
          [2.9116e+02, 2.1144e+02, 2.9851e+02, 2.2247e+02],
          [2.8947e+02, 1.6374e+02, 2.9682e+02, 1.7217e+02],
          [2.8903e+02, 1.1523e+02, 2.9605e+02, 1.2452e+02],
          [2.8676e+02, 3.3481e-01, 3.0356e+02, 1.3682e+01],
          [2.9167e+02, 1.7897e+02, 2.9898e+02, 1.8898e+02],
          [2.8700e+02, 2.0954e+01, 2.9466e+02, 3.0197e+01],
          [0.0000e+00, 2.0468e+02, 8.8145e+01, 3.2000e+02],
          [2.9084e+02, 1.9506e+02, 2.9750e+02, 2.0505e+02],
          [2.8459e+02, 2.4316e+02, 2.9224e+02, 2.5339e+02],
          [2.9226e+02, 5.4594e+00, 3.2000e+02, 5.6350e+01],
          [0.0000e+00, 1.8774e+01, 2.0305e+01, 3.3552e+01],
          [1.5775e+02, 2.5421e+

In [60]:
## Finetune 
!pip install torchinfo

In [62]:
# import torchsummary
from torchinfo import summary

In [95]:
import torchvision

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
# from models.model_summary import summary

def create_convnext_small_fasterrcnn_model(num_classes=81, pretrained=True, coco_model=False):
    # Load the pretrained features.
    if pretrained:
        backbone = torchvision.models.convnext_small(weights='DEFAULT').features
    else:
        backbone = torchvision.models.convnext_small().features

    # We need the output channels of the last convolutional layers from
    # the features for the Faster RCNN model.
    backbone.out_channels = 768

    # Generate anchors using the RPN. Here, we are using 5x3 anchors.
    # Meaning, anchors with 5 different sizes and 3 different aspect 
    # ratios.
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )

    # Feature maps to perform RoI cropping.
    # If backbone returns a Tensor, `featmap_names` is expected to
    # be [0]. We can choose which feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Final Faster RCNN model.
    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )

    return model
def create_convnext_large_fasterrcnn_model(num_classes=81, pretrained=True, coco_model=False):
    # Load the pretrained features.
    if pretrained:
        backbone = torchvision.models.convnext_large(weights='DEFAULT').features
    else:
        backbone = torchvision.models.convnext_large().features

    # We need the output channels of the last convolutional layers from
    # the features for the Faster RCNN model.
    backbone.out_channels = 1536

    # Generate anchors using the RPN. Here, we are using 5x3 anchors.
    # Meaning, anchors with 5 different sizes and 3 different aspect 
    # ratios.
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )

    # Feature maps to perform RoI cropping.
    # If backbone returns a Tensor, `featmap_names` is expected to
    # be [0]. We can choose which feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Final Faster RCNN model.
    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )

    return model
def create_efficientnet_b4_fasterrcnn_model(num_classes, pretrained=True, coco_model=False):
    # Load the pretrained EfficientNetB0 large features.
    backbone = torchvision.models.efficientnet_b4(pretrained=pretrained).features

    # We need the output channels of the last convolutional layers from
    # the features for the Faster RCNN model.
    backbone.out_channels = 1792

    # Generate anchors using the RPN. Here, we are using 5x3 anchors.
    # Meaning, anchors with 5 different sizes and 3 different aspect 
    # ratios.
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )

    # Feature maps to perform RoI cropping.
    # If backbone returns a Tensor, `featmap_names` is expected to
    # be [0]. We can choose which feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Final Faster RCNN model.
    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )

    return model
def create_resnet152_fasterrcnn_model(num_classes=81, pretrained=True, coco_model=False):
    model_backbone = torchvision.models.resnet152(weights='DEFAULT')

    conv1 = model_backbone.conv1
    bn1 = model_backbone.bn1
    relu = model_backbone.relu
    max_pool = model_backbone.maxpool
    layer1 = model_backbone.layer1
    layer2 = model_backbone.layer2
    layer3 = model_backbone.layer3
    layer4 = model_backbone.layer4

    backbone = nn.Sequential(
        conv1, bn1, relu, max_pool, 
        layer1, layer2, layer3, layer4
    )
    # We need the output channels of the last convolutional layers from
    # the features for the Faster RCNN model.
    # It is 960 for MobileNetV3.
    backbone.out_channels = 2048

    # Generate anchors using the RPN. Here, we are using 5x3 anchors.
    # Meaning, anchors with 5 different sizes and 3 different aspect 
    # ratios.
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )

    # Feature maps to perform RoI cropping.
    # If backbone returns a Tensor, `featmap_names` is expected to
    # be [0]. We can choose which feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Final Faster RCNN model.
    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )

    return model

In [86]:
# torchvision.models.regnet_x_8gf(weights='DEFAULT')

In [100]:
backbone = torchvision.models.maxvit_t(weights='DEFAULT').features

AttributeError: module 'torchvision.models' has no attribute 'maxvit_t'

In [71]:
# backbone = torchvision.models.regnet_x_8gf(weights='DEFAULT')
# features = backbone(torch.rand(1,3,320,320))
# print(features.shape)

torch.Size([1, 1000])


In [88]:
# # nn.Sequential(*(list(model.features.children())[:14]))
# for k,v in list(backbone.named_modules()):
#     print(k,v)

In [65]:
from collections import OrderedDict

backbone = torchvision.models.convnext_small(weights='DEFAULT').features

features = backbone(torch.rand(1,3,320,320))
# print(feat
if isinstance(features, torch.Tensor):
    features = OrderedDict([("0", features)])
x_filtered = []
_filtered = ['0','1','2','3']
for k, v in features.items():
    print(k)
    if k in _filtered:
        x_filtered.append(v)
[i.shape for i in x_filtered]

0


[torch.Size([1, 768, 10, 10])]

In [66]:
model = create_convnext_small_fasterrcnn_model(61)
model = model.eval()
x = model(torch.rand(1,3,320,320))
print(x)


[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}]


In [98]:
model = create_resnet152_fasterrcnn_model(61)
print(summary(model, input_size=(1, 3, 320, 320)))
model = create_efficientnet_b4_fasterrcnn_model(61)
print(summary(model, input_size=(1, 3, 320, 320)))
model = create_convnext_large_fasterrcnn_model(61)
print(summary(model, input_size=(1, 3, 320, 320)))

Layer (type:depth-idx)                             Output Shape              Param #
FasterRCNN                                         --                        --
├─GeneralizedRCNNTransform: 1-1                    --                        --
├─Sequential: 1-2                                  [1, 2048, 25, 25]         --
│    └─Conv2d: 2-1                                 [1, 64, 400, 400]         9,408
│    └─BatchNorm2d: 2-2                            [1, 64, 400, 400]         128
│    └─ReLU: 2-3                                   [1, 64, 400, 400]         --
│    └─MaxPool2d: 2-4                              [1, 64, 200, 200]         --
│    └─Sequential: 2-5                             [1, 256, 200, 200]        --
│    │    └─Bottleneck: 3-1                        [1, 256, 200, 200]        75,008
│    │    └─Bottleneck: 3-2                        [1, 256, 200, 200]        70,400
│    │    └─Bottleneck: 3-3                        [1, 256, 200, 200]        70,400
│    └─Sequential: 